# 📚 **Import Libraries**

In [11]:
import os
import requests
import logging

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression

In [12]:
logging.basicConfig(level=logging.INFO)

logging.getLogger("urllib3").setLevel(logging.WARNING)

# Functions

# 📡 **Data Acquisition**

## Download Data

In [13]:
try:
    if not os.path.exists("data"):
        os.mkdir("data")

    csv_url = "https://dsserver-prod-resources-1.s3.amazonaws.com/764/fires.csv"
    csv_path = "data/fires.csv"

    response = requests.get(csv_url)

    with open(csv_path, "wb") as csv_file:
        csv_file.write(response.content)

    fires = pd.read_csv(csv_path, index_col=0)

    logging.info("🟩 Download Success!")
except requests.exceptions.RequestException as req_error:
    logging.error("🟥 HTTP Error: %s", req_error)
except Exception as error:
    logging.error("🟥 Unexpected Error: %d", error)

INFO:root:🟩 Download Success!


## Read CSV

In [14]:
fires.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
1,7,5,mar,fri,86.2,26.2,94.3,5.1,NaN,51.0,6.7,0.0,0.0
2,7,4,oct,tue,90.6,NaN,669.1,6.7,18.0,33.0,0.9,0.0,0.0
3,7,4,oct,sat,90.6,43.7,NaN,6.7,14.6,33.0,1.3,0.0,0.0
4,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97.0,4.0,0.2,0.0
5,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99.0,NaN,0.0,0.0


# 🔍 **Data Exploration**

In [16]:
fires.info()

<class 'pandas.core.frame.DataFrame'>
Index: 517 entries, 1 to 517
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       517 non-null    int64  
 1   Y       517 non-null    int64  
 2   month   517 non-null    object 
 3   day     517 non-null    object 
 4   FFMC    469 non-null    float64
 5   DMC     496 non-null    float64
 6   DC      474 non-null    float64
 7   ISI     515 non-null    float64
 8   temp    496 non-null    float64
 9   RH      487 non-null    float64
 10  wind    482 non-null    float64
 11  rain    485 non-null    float64
 12  area    517 non-null    float64
dtypes: float64(9), int64(2), object(2)
memory usage: 56.5+ KB


In [19]:
pd.DataFrame(fires.isnull().sum(), columns=["null_count"])

,null_count
X,0
Y,0
month,0
day,0
FFMC,48
DMC,21
DC,43
ISI,2
temp,21
RH,30


In [15]:
fires.describe()

,X,Y,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
count,517.000000,517.000000,469.000000,496.000000,474.000000,515.000000,496.000000,487.000000,482.000000,485.000000,517.000000
mean,4.669246,4.299807,90.580384,111.195363,550.673418,9.018835,18.884677,44.381930,4.021784,0.023093,12.847292
std,2.313778,1.229900,5.698137,64.008450,246.061309,4.564890,5.748318,16.180372,1.794460,0.305532,63.655818
min,1.000000,2.000000,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000
25%,3.000000,4.000000,90.200000,70.800000,441.200000,6.450000,15.475000,33.000000,2.700000,0.000000,0.000000
50%,4.000000,4.000000,91.600000,108.300000,664.500000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000
75%,7.000000,5.000000,92.800000,141.575000,713.900000,10.750000,22.725000,53.500000,4.900000,0.000000,6.570000
max,9.000000,9.000000,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000


In [17]:
fires["month"].value_counts()

month
aug    184
sep    172
mar     54
jul     32
feb     20
jun     17
oct     15
apr      9
dec      9
jan      2
may      2
nov      1
Name: count, dtype: int64

In [18]:
fires["day"].value_counts()

day
sun    95
fri    85
sat    84
mon    74
tue    64
thu    61
wed    54
Name: count, dtype: int64

# 🧱 **Data Modeling**

# 🏗️ **Build Searcher**